In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/weather_bigdata_contest/code/

/content/drive/MyDrive/weather_bigdata_contest/code


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
# def mean_absolute_percentage_error(y_pred, y_true):
#     y_pred, y_true = np.array(y_pred), np.array(y_true)
#     return np.mean(np.abs((y_pred - y_true) / y_true)) * 100

In [4]:
wea_sale_ohe = pd.read_csv('../data/weather_sale_ohe.csv')
wea_sns_ohe = pd.read_csv('../data/weather_sns_ohe.csv')

In [ ]:
wea_sale_ohe.head(3)

,날짜,대분류,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과채 음료/주스,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 미스트,소분류_기초 화장용 오일/앰플,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_남성 로션,소분류_남성 메이크업,소분류_남성 선케어,소분류_남성 세트,소분류_남성 에센스,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트보조식,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_딸기우유,소분류_라디에이터,...,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_태닝용 선크림,소분류_토마토,소분류_파/양파,소분류_풋크림,소분류_프로폴리스/로얄젤리,소분류_해초류,소분류_핸드크림,소분류_허브차,소분류_헤어에센스,소분류_호박,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),20대 여성 판매량(개),20대 남성 판매량(개),30대 여성 판매량(개),30대 남성 판매량(개),40대 여성 판매량(개),40대 남성 판매량(개),50대 여성 판매량(개),50대 남성 판매량(개),60대 여성 판매량(개),60대 남성 판매량(개),일별 판매 합계량(개)
0,2018-01-01,냉난방가전,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,7.0
1,2018-01-01,냉난방가전,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,10.0,2.0,35.0,18.0,14.0,24.0,2.0,12.0,2.0,1.0,120.0
2,2018-01-01,냉난방가전,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,2.0,28.0,26.0,16.0,18.0,6.0,4.0,2.0,3.0,106.0


In [ ]:
wea_sns_ohe.head(3)

,날짜,대분류,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과채 음료/주스,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 미스트,소분류_기초 화장용 오일/앰플,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_남성 로션,소분류_남성 메이크업,소분류_남성 선케어,소분류_남성 세트,소분류_남성 에센스,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트보조식,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_딸기우유,소분류_라디에이터,...,소분류_전기장판,소분류_전통차,소분류_절임배추/김치속,소분류_제습기,소분류_조개,소분류_차/곡물 음료,소분류_참외/메론/수박,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_태닝용 선크림,소분류_토마토,소분류_파/양파,소분류_풋크림,소분류_프로폴리스/로얄젤리,소분류_해초류,소분류_핸드크림,소분류_허브차,소분류_헤어에센스,소분류_호박,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),SNS언급량
0,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.895782
1,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,88.715916
2,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,18.342442


## ML으로 연령, 성별 카테고리별 판매량 예측, SNS 언급량 예측

In [5]:
def sale_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()
    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)
    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'20대 여성 판매량(개)':'60대 남성 판매량(개)'].values])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    # X_train, y_train 생성

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-10]
    y_train = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-10]])
      y_train = np.vstack([y_train, df[day-1:, -10:]])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-10]
    y_test = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-10]])
      y_test = np.vstack([y_test, df[day-1:, -10:]])

    return X_train, X_test, y_train, y_test

In [6]:
def sns_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)

    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'SNS언급량'].values.reshape(-1, 1)])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-1]
    y_train = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-1]])
      y_train = np.vstack([y_train, df[day-1:, -1].reshape(-1,1)])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-1]
    y_test = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-1]])
      y_test = np.vstack([y_test, df[day-1:, -1].reshape(-1,1)])

    return X_train, X_test, y_train.reshape(-1), y_test.reshape(-1)

### 1) Linear Regression

#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [7]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [8]:
preds =[]

for i in range(10):
    ln_reg = LinearRegression()
    ln_reg.fit(X_train, y_train[:, i])
    pred = ln_reg.predict(X_test)
    preds.append(pred)

In [9]:
linear_sale_mse = mean_squared_error(y_test, np.array(preds).T)
linear_sale_mae = mean_absolute_error(y_test, np.array(preds).T)
print('LinearRegression의 판매량 예측 MSE: ', linear_sale_mse)
print('LinearRegression의 판매량 예측 RMSE: ', np.sqrt(linear_sale_mse))
print('LinearRegression의 판매량 예측 MAE: ', linear_sale_mae)

LinearRegression의 판매량 예측 MSE:  468.12446311908377
LinearRegression의 판매량 예측 RMSE:  21.63618411640749
LinearRegression의 판매량 예측 MAE:  8.318945498408143


#### (2) SNS 언급량 예측

In [10]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [11]:
ln_reg = LinearRegression()
ln_reg.fit(X_train, y_train)
pred = ln_reg.predict(X_test)

linear_sns_mse = mean_squared_error(pred, y_test)
linear_sns_mae = mean_absolute_error(pred, y_test)
print('LinearRegression의 SNS 언급량 MSE:', linear_sns_mse)
print('LinearRegression의 SNS 언급량 RMSE:', np.sqrt(linear_sns_mse))
print('LinearRegression의 SNS 언급량 MAE:', linear_sns_mae)

LinearRegression의 SNS 언급량 MSE: 817.3566183174156
LinearRegression의 SNS 언급량 RMSE: 28.589449423124883
LinearRegression의 SNS 언급량 MAE: 12.095126085703644


### 2) Ridge Regression

#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [12]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [13]:
preds =[]

for i in range(10):
    rg_reg = Ridge(alpha = 0.05, normalize = True)
    rg_reg.fit(X_train, y_train[:, i])
    pred = rg_reg.predict(X_test)
    preds.append(pred)

ridge_sale_mse = mean_squared_error(y_test, np.array(preds).T)
ridge_sale_mae = mean_absolute_error(y_test, np.array(preds).T)
print('RidgeRegression의 판매량 예측 MSE: ', ridge_sale_mse)
print('RidgeRegression의 판매량 예측 RMSE: ', np.sqrt(ridge_sale_mse))
print('RidgeRegression의 판매량 예측 MAE: ', ridge_sale_mae)


RidgeRegression의 판매량 예측 MSE:  476.77577284844773
RidgeRegression의 판매량 예측 RMSE:  21.835195736435423
RidgeRegression의 판매량 예측 MAE:  8.323271880381688


#### (2) SNS 언급량 예측

In [14]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [15]:
rg_reg = Ridge(alpha = 0.05, normalize = True)
rg_reg.fit(X_train, y_train)
pred = rg_reg.predict(X_test)

ridge_sns_mse = mean_squared_error(pred, y_test)
ridge_sns_mae = mean_absolute_error(pred, y_test)
print('RidgeRegression의 SNS 언급량 MSE:', ridge_sns_mse)
print('RidgeRegression의 SNS 언급량 RMSE:', np.sqrt(ridge_sns_mse))
print('RidgeRegression의 SNS 언급량 MAE:', ridge_sns_mae)

RidgeRegression의 SNS 언급량 MSE: 744.664300718702
RidgeRegression의 SNS 언급량 RMSE: 27.288537899981044
RidgeRegression의 SNS 언급량 MAE: 12.28204240654706


### 3) Lasso Regression

#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [16]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [17]:
preds =[]

for i in range(10):
    ls_reg = Lasso(alpha = 0.01, normalize = True)
    ls_reg.fit(X_train, y_train[:, i])
    pred = ls_reg.predict(X_test)
    preds.append(pred)

lasso_sale_mse = mean_squared_error(y_test, np.array(preds).T)
lasso_sale_mae = mean_absolute_error(y_test, np.array(preds).T)
print('LassoRegression의 판매량 예측 MSE: ', lasso_sale_mse)
print('LassoRegression의 판매량 예측 RMSE: ', np.sqrt(lasso_sale_mse))
print('LassoRegression의 판매량 예측 MAE: ', lasso_sale_mae)

LassoRegression의 판매량 예측 MSE:  643.6209424213383
LassoRegression의 판매량 예측 RMSE:  25.36968550103328
LassoRegression의 판매량 예측 MAE:  13.078651748210623


#### (2) SNS 언급량 예측

In [18]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [19]:
ls_reg = Lasso(alpha = 0.01, normalize = True)
ls_reg.fit(X_train, y_train)
pred = ls_reg.predict(X_test)

lasso_sns_mse = mean_squared_error(pred, y_test)
lasso_sns_mae = mean_absolute_error(pred, y_test)
print('LassoRegression의 SNS 언급량 MSE:', lasso_sns_mse)
print('LassoRegression의 SNS 언급량 RMSE:', np.sqrt(lasso_sns_mse))
print('LassoRegression의 SNS 언급량 MAE:', lasso_sns_mae)


LassoRegression의 SNS 언급량 MSE: 835.9535325395336
LassoRegression의 SNS 언급량 RMSE: 28.912861023072995
LassoRegression의 SNS 언급량 MAE: 18.480135175502724


### 4) XGBRegressor




#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [20]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [21]:
preds = []
for i in range(10):
    xgb_reg = XGBRegressor(n_estimators = 300)
    eval_set = [(X_test, y_test[:, i])]
    xgb_reg.fit(X_train, y_train[:, i], early_stopping_rounds = 100, eval_metric = 'rmse', eval_set = eval_set, verbose = True)
    pred = xgb_reg.predict(X_test)
    preds.append(pred)

xgb_sale_mse = mean_squared_error(np.array(preds).T, y_test)
xgb_sale_mae = mean_absolute_error(np.array(preds).T, y_test)
print('XGBRegressor의 판매량 예측 MSE:', xgb_sale_mse)
print('XGBRegressor의 판매량 예측 RMSE:', np.sqrt(xgb_sale_mse))
print('XGBRegressor의 판매량 예측 MSE:', xgb_sale_mae)

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[12:54:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:56.5965
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:52.9135
[2]	validation_0-rmse:49.7087
[3]	validation_0-rmse:46.906
[4]	validation_0-rmse:44.5963
[5]	validation_0-rmse:42.5393
[6]	validation_0-rmse:40.7069
[7]	validation_0-rmse:39.4481
[8]	validation_0-rmse:38.2783
[9]	validation_0-rmse:37.0715
[10]	validation_0-rmse:36.1642
[11]	validation_0-rmse:35.1773
[12]	validation_0-rmse:34.5841
[13]	validation_0-rmse:33.9912
[14]	validation_0-rmse:33.3357
[15]	validation_0-rmse:32.8367
[16]	validation_0-rmse:32.2487
[17]	validation_0-rmse:31.8986
[18]	validation_0-rmse:31.5756
[19]	validation_0-rmse:31.0712
[20]	validation_0-rmse:30.8401
[21]	validation_0-rmse:30.5322
[22]	validation_0-rmse:30.134
[23]	validation_0-rmse:29.9204
[24]	validation_0-rmse:29.6892
[25]	validation_0-rmse:29.4657
[26]	validati

#### (2) SNS 언급량 예측

In [22]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [41]:
xgb_reg = XGBRegressor(n_estimators = 300)
eval_set = [(X_test, y_test)]
xgb_reg.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = 'rmse', eval_set = eval_set, verbose = True)
pred = xgb_reg.predict(X_test)

xgb_sns_mse = mean_squared_error(pred, y_test)
xgb_sns_mae = mean_absolute_error(pred, y_test)
print('XGBRegressor의 SNS 언급량 MSE:', xgb_sns_mse)
print('XGBRegressor의 SNS 언급량 RMSE:', np.sqrt(xgb_sns_mse))
print('XGBRegressor의 SNS 언급량 MAE:', xgb_sns_mae)


[13:10:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:166.259
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:152.19
[2]	validation_0-rmse:139.694
[3]	validation_0-rmse:128.456
[4]	validation_0-rmse:118.799
[5]	validation_0-rmse:110.347
[6]	validation_0-rmse:102.804
[7]	validation_0-rmse:96.0495
[8]	validation_0-rmse:90.3516
[9]	validation_0-rmse:85.2837
[10]	validation_0-rmse:80.7741
[11]	validation_0-rmse:76.9692
[12]	validation_0-rmse:73.4583
[13]	validation_0-rmse:70.7121
[14]	validation_0-rmse:67.9335
[15]	validation_0-rmse:65.7727
[16]	validation_0-rmse:63.6334
[17]	validation_0-rmse:62.0707
[18]	validation_0-rmse:60.3556
[19]	validation_0-rmse:59.092
[20]	validation_0-rmse:57.7133
[21]	validation_0-rmse:56.5635
[22]	validation_0-rmse:55.5968
[23]	validation_0-rmse:54.5377
[24]	validation_0-rmse:53.5122
[25]	validation_0-rmse:52.8196
[26]	validati

### 5) LGBMRegressor




#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [25]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [28]:
lgbm_reg = LGBMRegressor(n_estimators = 300)
mo_reg_lgbm = MultiOutputRegressor(lgbm_reg)

eval_set = [(X_test, y_test)]

mo_reg_lgbm.fit(X_train, y_train)

pred = mo_reg_lgbm.predict(X_test)

lgbm_sale_mse = mean_squared_error(y_test, pred)
lgbm_sale_mae = mean_absolute_error(y_test, pred)
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mse)
print('LGBMRegressor의 판매량 예측 RMSE: ', np.sqrt(lgbm_sale_mse))
print('LGBMRegressor의 판매량 예측 MAE: ', lgbm_sale_mae)

LGBMRegressor의 판매량 예측 MSE:  439.5142314168567
LGBMRegressor의 판매량 예측 RMSE:  20.96459471148576
LGBMRegressor의 판매량 예측 MAE:  7.958411629060916


#### (2) SNS 언급량 예측

In [39]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [42]:
lgbm_reg = LGBMRegressor(n_estimators = 300)
eval_set = [(X_test, y_test)]
lgbm_reg.fit(X_train, y_train, eval_metric = 'mean_squared_error',
             eval_set = eval_set, verbose = False)
lgbm_sns_pred = lgbm_reg.predict(X_test)

lgbm_sns_mse = mean_squared_error(lgbm_sns_pred, y_test)
lgbm_sns_mae = mean_absolute_error(lgbm_sns_pred, y_test)
print('LGBMRegressor의 SNS 언급량 예측 MSE:', lgbm_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(lgbm_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE:', lgbm_sns_mae)

LGBMRegressor의 SNS 언급량 예측 MSE: 674.1798275828507
LGBMRegressor의 SNS 언급량 예측 RMSE: 25.964973090354835
LGBMRegressor의 SNS 언급량 예측 MAE: 10.479638969313406


### 6) CatBoostRegressor



In [31]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 4.8 kB/s 



#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [32]:
from catboost import CatBoostRegressor

In [33]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [34]:
cb_reg = CatBoostRegressor(n_estimators = 300)
mo_reg_cb = MultiOutputRegressor(cb_reg)

eval_set = [(X_test, y_test)]

mo_reg_cb.fit(X_train, y_train)

pred = mo_reg_cb.predict(X_test)

cb_sale_mse = mean_squared_error(y_test, pred)
cb_sale_mae = mean_absolute_error(y_test, pred)
print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sale_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sale_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sale_mae)

Learning rate set to 0.20735
0:	learn: 48.3342575	total: 60.7ms	remaining: 18.2s
1:	learn: 42.8550560	total: 73ms	remaining: 10.9s
2:	learn: 38.9694495	total: 94.7ms	remaining: 9.38s
3:	learn: 36.2307462	total: 106ms	remaining: 7.82s
4:	learn: 34.1473449	total: 116ms	remaining: 6.86s
5:	learn: 32.6215383	total: 126ms	remaining: 6.19s
6:	learn: 31.4767886	total: 139ms	remaining: 5.83s
7:	learn: 30.6037425	total: 150ms	remaining: 5.47s
8:	learn: 29.8122938	total: 160ms	remaining: 5.18s
9:	learn: 29.2519906	total: 171ms	remaining: 4.96s
10:	learn: 28.6980474	total: 181ms	remaining: 4.76s
11:	learn: 28.1998437	total: 192ms	remaining: 4.61s
12:	learn: 27.7657453	total: 203ms	remaining: 4.48s
13:	learn: 27.3682571	total: 213ms	remaining: 4.36s
14:	learn: 27.0158022	total: 224ms	remaining: 4.26s
15:	learn: 26.7106099	total: 235ms	remaining: 4.17s
16:	learn: 26.3680864	total: 246ms	remaining: 4.09s
17:	learn: 26.0609905	total: 261ms	remaining: 4.09s
18:	learn: 25.7731861	total: 272ms	remaining

#### (2) SNS 언급량 예측

In [35]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [36]:
cb_reg = CatBoostRegressor(n_estimators = 300)
eval_set = [(X_test, y_test)]
cb_reg.fit(X_train, y_train, early_stopping_rounds = 100, verbose = True)
pred = cb_reg.predict(X_test)

cb_sns_mse = mean_squared_error(pred, y_test)
cb_sns_mae = mean_absolute_error(pred, y_test)
print('CatBoostRegressor의 SNS 언급량 예측 MSE:', cb_sns_mse)
print('CatBoostRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(cb_sns_mse))
print('CatBoostRegressor의 SNS 언급량 예측 MAE:', cb_sns_mae)


Learning rate set to 0.208218
0:	learn: 150.8000343	total: 12.7ms	remaining: 3.81s
1:	learn: 125.8514785	total: 22.8ms	remaining: 3.4s
2:	learn: 106.6706386	total: 33.1ms	remaining: 3.27s
3:	learn: 92.1257699	total: 43.7ms	remaining: 3.23s
4:	learn: 80.8982657	total: 54ms	remaining: 3.18s
5:	learn: 72.4510310	total: 64.1ms	remaining: 3.14s
6:	learn: 65.8879266	total: 74.5ms	remaining: 3.12s
7:	learn: 60.7861372	total: 84.8ms	remaining: 3.1s
8:	learn: 56.8545427	total: 95.7ms	remaining: 3.1s
9:	learn: 53.7361192	total: 106ms	remaining: 3.07s
10:	learn: 51.1800554	total: 116ms	remaining: 3.05s
11:	learn: 49.1358576	total: 127ms	remaining: 3.04s
12:	learn: 47.3674038	total: 141ms	remaining: 3.1s
13:	learn: 45.7206247	total: 151ms	remaining: 3.09s
14:	learn: 44.3540990	total: 162ms	remaining: 3.08s
15:	learn: 43.1737506	total: 173ms	remaining: 3.06s
16:	learn: 41.9847180	total: 183ms	remaining: 3.05s
17:	learn: 41.0257864	total: 194ms	remaining: 3.04s
18:	learn: 40.1338940	total: 204ms	rem

### 요약

#### 1) 모델별 판매량 예측

In [43]:
print('LinearRegression의 판매량 예측 MSE: ', linear_sale_mse)
print('LinearRegression의 판매량 예측 RMSE: ', np.sqrt(linear_sale_mse))
print('LinearRegression의 판매량 예측 MAE: ', linear_sale_mae)
print('------------------------------------------------------')
print('RidgeRegression의 판매량 예측 MSE: ', ridge_sale_mse)
print('RidgeRegression의 판매량 예측 RMSE: ', np.sqrt(ridge_sale_mse))
print('RidgeRegression의 판매량 예측 MAE: ', ridge_sale_mae)
print('------------------------------------------------------')
print('LassoRegression의 판매량 예측 MSE: ', lasso_sale_mse)
print('LassoRegression의 판매량 예측 RMSE: ', np.sqrt(lasso_sale_mse))
print('LassoRegression의 판매량 예측 MSE: ', lasso_sale_mae)
print('------------------------------------------------------')
print('XGBRegressor의 판매량 예측 MSE:', xgb_sale_mse)
print('XGBRegressor의 판매량 예측 RMSE:', np.sqrt(xgb_sale_mse))
print('XGBRegressor의 판매량 예측 MSE:', xgb_sale_mae)
print('------------------------------------------------------')
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mse)
print('LGBMRegressor의 판매량 예측 RMSE: ', np.sqrt(lgbm_sale_mse))
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mae)
print('------------------------------------------------------')
print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sale_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sale_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sale_mae)

LinearRegression의 판매량 예측 MSE:  468.12446311908377
LinearRegression의 판매량 예측 RMSE:  21.63618411640749
LinearRegression의 판매량 예측 MAE:  8.318945498408143
------------------------------------------------------
RidgeRegression의 판매량 예측 MSE:  476.77577284844773
RidgeRegression의 판매량 예측 RMSE:  21.835195736435423
RidgeRegression의 판매량 예측 MAE:  8.323271880381688
------------------------------------------------------
LassoRegression의 판매량 예측 MSE:  643.6209424213383
LassoRegression의 판매량 예측 RMSE:  25.36968550103328
LassoRegression의 판매량 예측 MSE:  13.078651748210623
------------------------------------------------------
XGBRegressor의 판매량 예측 MSE: 459.42876601404043
XGBRegressor의 판매량 예측 RMSE: 21.43428949170092
XGBRegressor의 판매량 예측 MSE: 9.15863685163596
------------------------------------------------------
LGBMRegressor의 판매량 예측 MSE:  439.5142314168567
LGBMRegressor의 판매량 예측 RMSE:  20.96459471148576
LGBMRegressor의 판매량 예측 MSE:  7.958411629060916
------------------------------------------------------
CatBoostReg

#### 2) 모델별 SNS 언급량 예측

In [44]:
print('LinearRegression의 SNS 언급량 MSE:', linear_sns_mse)
print('LinearRegression의 SNS 언급량 RMSE:', np.sqrt(linear_sns_mse))
print('LinearRegression의 SNS 언급량 MAE:', linear_sns_mae)
print('------------------------------------------------------')
print('RidgeRegression의 SNS 언급량 MSE:', ridge_sns_mse)
print('RidgeRegression의 SNS 언급량 RMSE:', np.sqrt(ridge_sns_mse))
print('RidgeRegression의 SNS 언급량 MAE:', ridge_sns_mae)
print('------------------------------------------------------')
print('LassoRegression의 SNS 언급량 MSE:', lasso_sns_mse)
print('LassoRegression의 SNS 언급량 RMSE:', np.sqrt(lasso_sns_mse))
print('LassoRegression의 SNS 언급량 MAE:', lasso_sns_mae)
print('------------------------------------------------------')
print('XGBRegressor의 SNS 언급량 MSE:', xgb_sns_mse)
print('XGBRegressor의 SNS 언급량 RMSE:', np.sqrt(xgb_sns_mse))
print('XGBRegressor의 SNS 언급량 MAE:', xgb_sns_mae)
print('------------------------------------------------------')
print('LGBMRegressor의 SNS 언급량 예측 MSE:', lgbm_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(lgbm_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE:', lgbm_sns_mae)
print('------------------------------------------------------')
print('CatBoostRegressor의 SNS 언급량 예측 MSE:', cb_sns_mse)
print('CatBoostRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(cb_sns_mse))
print('CatBoostRegressor의 SNS 언급량 예측 MAE:', cb_sns_mae)

LinearRegression의 SNS 언급량 MSE: 817.3566183174156
LinearRegression의 SNS 언급량 RMSE: 28.589449423124883
LinearRegression의 SNS 언급량 MAE: 12.095126085703644
------------------------------------------------------
RidgeRegression의 SNS 언급량 MSE: 744.664300718702
RidgeRegression의 SNS 언급량 RMSE: 27.288537899981044
RidgeRegression의 SNS 언급량 MAE: 12.28204240654706
------------------------------------------------------
LassoRegression의 SNS 언급량 MSE: 835.9535325395336
LassoRegression의 SNS 언급량 RMSE: 28.912861023072995
LassoRegression의 SNS 언급량 MAE: 18.480135175502724
------------------------------------------------------
XGBRegressor의 SNS 언급량 MSE: 626.7383655064738
XGBRegressor의 SNS 언급량 RMSE: 25.0347431683745
XGBRegressor의 SNS 언급량 MAE: 13.337922119383311
------------------------------------------------------
LGBMRegressor의 SNS 언급량 예측 MSE: 674.1798275828507
LGBMRegressor의 SNS 언급량 예측 RMSE: 25.964973090354835
LGBMRegressor의 SNS 언급량 예측 MAE: 10.479638969313406
----------------------------------------------------